In [ ]:
import json
import timeit, time
from functools import partial
from matplotlib import pyplot
import numpy as np
from pathlib import Path
import math

# Not Your Grandma's Review Analyzer

## COSC 320 - Milestone 3

### Team Members

Quinn Marshall, Archita Gattani, and Connor Doman

## Algorithm

Our algorithm requires us to read from a JSON file. We load that here.

In [ ]:
DATA_FILE_FOLDER = '../data/raw/'
DATA_FILE_NAME = 'sports_and_outdoors_reviews'
DATA_FILE_TYPE = '.json'
DATA_FILE_PATH = Path(DATA_FILE_FOLDER + DATA_FILE_NAME + DATA_FILE_TYPE)

def get_reviews_from_sports_data():    
    sports_data = []
    start = time.time()
    with open(Path(DATA_FILE_FOLDER + DATA_FILE_NAME + '_million' + DATA_FILE_TYPE), 'r') as f:
        sports_data = json.load(f)
    end = time.time()

    print(f"Got {len(sports_data)} reviews in {end - start:.2f}s")

    total_review_size = 0
    review_text = []
    for i, r in enumerate(sports_data):
        if 'reviewText' in r:
            total_review_size += len(r['reviewText'])
            review_text.append(r['reviewText'])
    print(f"Average review length: {math.ceil(total_review_size / len(review_text))} characters")

    return review_text

In [ ]:

REPLACEMENT_DICTIONARY = {
    "the": "THE",
}

def replace_with_new_word(text):
    text = text.strip()

    # splitting the words in T
    text_array = []
    buffer_word = ""
    for char in text:
        
        if char == " ":
            text_array.append(buffer_word)
            buffer_word = ""
        else:
            buffer_word += char
    text_array.append(buffer_word)

    # replacing the words in T
    for old_word, replacement in REPLACEMENT_DICTIONARY.items():
        for word in text_array:
            if word == old_word:
                text_array[text_array.index(word)] = replacement
    
    # rebuilding the string
    text = ""
    for word in text_array:
        text += word + " "

    return text